In [1]:
try:
    get_ipython().run_line_magic('load_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')
    get_ipython().run_line_magic('matplotlib', 'qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.source_extraction.cnmf.initialization import init_neurons_corr_pnr
import cv2
import pickle


try:
    cv2.setNumThreads(0)
except:
    pass

#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

        2146 [__init__.py:            <module>():47] [11868] Creating converter from 7 to 5
        2152 [__init__.py:            <module>():47] [11868] Creating converter from 5 to 7
        2154 [__init__.py:            <module>():47] [11868] Creating converter from 7 to 5
        2155 [__init__.py:            <module>():47] [11868] Creating converter from 5 to 7
       13103 [tpu_cluster_resolver.py:            <module>():32] [11868] Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
       16181 [cluster.py:       setup_cluster():411] [11868] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman


### File name list

In [2]:
fnames = [r'H:\CM2scope_experimental_data\trace_fear_conditioning\train\gzc_rasgrf-ai148d-369\My_V4_Miniscope\denoise\test_data\AMF-512.tif'.replace('\\','\\'),
         ]
print(fnames)

['H:\\CM2scope_experimental_data\\trace_fear_conditioning\\train\\gzc_rasgrf-ai148d-369\\My_V4_Miniscope\\denoise\\test_data\\AMF-512.tif']


### Motion Correction

In [3]:
# dataset dependent parameters
frate = 10                      # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = True         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (5, 5)      # size of high pass spatial filtering, used in 1p data
max_shifts = (20, 20)      # maximum allowed rigid shift
strides = (128, 128)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (32, 32)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 10  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries
num_frames_split = 20  #根据数据量大大小进行更改

mc_dict = {
    'fnames': [fnames[0]],
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan,
    'num_frames_split':num_frames_split
}
opts = params.CNMFParams(params_dict=mc_dict)
for fname in fnames:
    opts.change_params(params_dict={'fnames': [fname]})

    # do motion correction rigid
    mc = MotionCorrect([fname], dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    fname_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                     np.max(np.abs(mc.y_shifts_els)))).astype(int)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(int)
        plt.subplot(1, 2, 1); plt.imshow(mc.total_template_rig)  # % plot template
        plt.subplot(1, 2, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
        plt.legend(['x shifts', 'y shifts'])
        plt.xlabel('frames')
        plt.ylabel('pixels')

    motion_corrected_tif_name = fname.split(".")[0]+'_MC.tif'
    F_frames=cm.load(fname_mc[0],outtype=np.float16)
    F_frames.astype(np.uint8).save(motion_corrected_tif_name,to32=False,imagej=True,bigtiff=False)

    name = fname.split(".")[0]+'_motion_corrected.pkl'
    # 创建一个字典来保存实例的属性
    attributes = {attr: getattr(mc, attr) for attr in dir(mc) 
                  if not callable(getattr(mc, attr)) 
                  and not attr.startswith("__") 
                  and attr != 'dview'}
    # 使用pickle将属性字典保存为文件
    with open(name, 'wb') as file:
        pickle.dump(attributes, file)

       17457 [params.py:                 set():977] [11868] Changing key fnames in group data from None to ['H:\\CM2scope_experimental_data\\trace_fear_conditioning\\train\\gzc_rasgrf-ai148d-369\\My_V4_Miniscope\\denoise\\test_data\\AMF-512.tif']
       17459 [params.py:                 set():977] [11868] Changing key fr in group data from 30 to 10
       17460 [params.py:                 set():977] [11868] Changing key pw_rigid in group motion from False to True
       17462 [params.py:                 set():977] [11868] Changing key max_shifts in group motion from (6, 6) to (20, 20)
       17463 [params.py:                 set():977] [11868] Changing key gSig_filt in group motion from None to (5, 5)
       17465 [params.py:                 set():977] [11868] Changing key strides in group motion from (96, 96) to (128, 128)
       17468 [params.py:                 set():977] [11868] Changing key max_deviation_rigid in group motion from 3 to 10
       17470 [params.py:                 s

     2476269 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [4398 4399 4400 4401 4402 4403 4404 4405 4406 4407 4408 4409 4410 4411
 4412 4413 4414 4415 4416 4417 4418 4419 4420 4421 4422 4423 4424 4425
 4426 4427 4428 4429 4430 4431 4432 4433 4434 4435 4436 4437 4438 4439
 4440 4441 4442 4443 4444 4445 4446 4447 4448 4449 4450 4451 4452 4453
 4454 4455 4456 4457 4458 4459 4460 4461 4462 4463 4464 4465 4466 4467
 4468 4469 4470 4471 4472 4473 4474 4475 4476 4477 4478 4479 4480 4481
 4482 4483 4484 4485 4486 4487 4488 4489 4490 4491 4492 4493 4494 4495
 4496 4497 4498 4499 4500 4501 4502 4503 4504 4505 4506 4507 4508 4509
 4510 4511 4512 4513 4514 4515 4516 4517 4518 4519 4520 4521 4522 4523
 4524 4525 4526 4527 4528 4529 4530 4531 4532 4533 4534 4535 4536 4537
 4538 4539 4540 4541 4542 4543 4544 4545 4546 4547 4548 4549 4550 4551
 4552 4553 4554 4555 4556 4557 4558 4559 4560 4561 4562 4563 4564 4565
 4566]
     2476271 [motion_correction.py:motion_c

     3159097 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259
 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313
 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331
 332 333 334 335 336 337 338 339]
     3159099 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357
 358 359 360 361 362 363 364 365 366 367

     3159124 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876
 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890
 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904
 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918
 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932
 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946
 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960
 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974
 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988
 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030
 2031]
     3159126 [motion_correction.py:motion_c

     3159147 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [3384 3385 3386 3387 3388 3389 3390 3391 3392 3393 3394 3395 3396 3397
 3398 3399 3400 3401 3402 3403 3404 3405 3406 3407 3408 3409 3410 3411
 3412 3413 3414 3415 3416 3417 3418 3419 3420 3421 3422 3423 3424 3425
 3426 3427 3428 3429 3430 3431 3432 3433 3434 3435 3436 3437 3438 3439
 3440 3441 3442 3443 3444 3445 3446 3447 3448 3449 3450 3451 3452 3453
 3454 3455 3456 3457 3458 3459 3460 3461 3462 3463 3464 3465 3466 3467
 3468 3469 3470 3471 3472 3473 3474 3475 3476 3477 3478 3479 3480 3481
 3482 3483 3484 3485 3486 3487 3488 3489 3490 3491 3492 3493 3494 3495
 3496 3497 3498 3499 3500 3501 3502 3503 3504 3505 3506 3507 3508 3509
 3510 3511 3512 3513 3514 3515 3516 3517 3518 3519 3520 3521 3522 3523
 3524 3525 3526 3527 3528 3529 3530 3531 3532 3533 3534 3535 3536 3537
 3538 3539 3540 3541 3542 3543 3544 3545 3546 3547 3548 3549 3550 3551
 3552]
     3159149 [motion_correction.py:motion_c

...

    21958055 [motion_correction.py:motion_correction_piecewise():3296] [2088] ** Starting parallel motion correction **
    23006534 [motion_correction.py:motion_correction_piecewise():3304] [2088] ** Finished parallel motion correction **
    23014170 [motion_correction.py:motion_correct_batch_pwrigid():2984] [2088] Adding to movie 3.2461562
    23014172 [motion_correction.py:motion_correct_batch_pwrigid():2987] [2088] 0
    23014184 [motion_correction.py:motion_correct_batch_pwrigid():2997] [2088] saving mmap of E:\CM2scope\social\gzc_rasgrf-ai148d-371\My_V4_Miniscope\AMF.tif
    23014188 [motion_correction.py:motion_correction_piecewise():3249] [2088] Number of Splits: 33
    23014201 [motion_correction.py:motion_correction_piecewise():3283] [2088] Saving file as E:\CM2scope\social\gzc_rasgrf-ai148d-371\My_V4_Miniscope\AMF_els__d1_1944_d2_2592_d3_1_order_F_frames_5580.mmap
    23014204 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [  0   1   2 

    23014224 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [1524 1525 1526 1527 1528 1529 1530 1531 1532 1533 1534 1535 1536 1537
 1538 1539 1540 1541 1542 1543 1544 1545 1546 1547 1548 1549 1550 1551
 1552 1553 1554 1555 1556 1557 1558 1559 1560 1561 1562 1563 1564 1565
 1566 1567 1568 1569 1570 1571 1572 1573 1574 1575 1576 1577 1578 1579
 1580 1581 1582 1583 1584 1585 1586 1587 1588 1589 1590 1591 1592 1593
 1594 1595 1596 1597 1598 1599 1600 1601 1602 1603 1604 1605 1606 1607
 1608 1609 1610 1611 1612 1613 1614 1615 1616 1617 1618 1619 1620 1621
 1622 1623 1624 1625 1626 1627 1628 1629 1630 1631 1632 1633 1634 1635
 1636 1637 1638 1639 1640 1641 1642 1643 1644 1645 1646 1647 1648 1649
 1650 1651 1652 1653 1654 1655 1656 1657 1658 1659 1660 1661 1662 1663
 1664 1665 1666 1667 1668 1669 1670 1671 1672 1673 1674 1675 1676 1677
 1678 1679 1680 1681 1682 1683 1684 1685 1686 1687 1688 1689 1690 1691
 1692]
    23014227 [motion_correction.py:motion_c

    23014247 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [3045 3046 3047 3048 3049 3050 3051 3052 3053 3054 3055 3056 3057 3058
 3059 3060 3061 3062 3063 3064 3065 3066 3067 3068 3069 3070 3071 3072
 3073 3074 3075 3076 3077 3078 3079 3080 3081 3082 3083 3084 3085 3086
 3087 3088 3089 3090 3091 3092 3093 3094 3095 3096 3097 3098 3099 3100
 3101 3102 3103 3104 3105 3106 3107 3108 3109 3110 3111 3112 3113 3114
 3115 3116 3117 3118 3119 3120 3121 3122 3123 3124 3125 3126 3127 3128
 3129 3130 3131 3132 3133 3134 3135 3136 3137 3138 3139 3140 3141 3142
 3143 3144 3145 3146 3147 3148 3149 3150 3151 3152 3153 3154 3155 3156
 3157 3158 3159 3160 3161 3162 3163 3164 3165 3166 3167 3168 3169 3170
 3171 3172 3173 3174 3175 3176 3177 3178 3179 3180 3181 3182 3183 3184
 3185 3186 3187 3188 3189 3190 3191 3192 3193 3194 3195 3196 3197 3198
 3199 3200 3201 3202 3203 3204 3205 3206 3207 3208 3209 3210 3211 3212
 3213]
    23014249 [motion_correction.py:motion_c

    23014267 [motion_correction.py:motion_correction_piecewise():3289] [2088] Processing: frames: [4566 4567 4568 4569 4570 4571 4572 4573 4574 4575 4576 4577 4578 4579
 4580 4581 4582 4583 4584 4585 4586 4587 4588 4589 4590 4591 4592 4593
 4594 4595 4596 4597 4598 4599 4600 4601 4602 4603 4604 4605 4606 4607
 4608 4609 4610 4611 4612 4613 4614 4615 4616 4617 4618 4619 4620 4621
 4622 4623 4624 4625 4626 4627 4628 4629 4630 4631 4632 4633 4634 4635
 4636 4637 4638 4639 4640 4641 4642 4643 4644 4645 4646 4647 4648 4649
 4650 4651 4652 4653 4654 4655 4656 4657 4658 4659 4660 4661 4662 4663
 4664 4665 4666 4667 4668 4669 4670 4671 4672 4673 4674 4675 4676 4677
 4678 4679 4680 4681 4682 4683 4684 4685 4686 4687 4688 4689 4690 4691
 4692 4693 4694 4695 4696 4697 4698 4699 4700 4701 4702 4703 4704 4705
 4706 4707 4708 4709 4710 4711 4712 4713 4714 4715 4716 4717 4718 4719
 4720 4721 4722 4723 4724 4725 4726 4727 4728 4729 4730 4731 4732 4733
 4734]
    23014269 [motion_correction.py:motion_c

    24235593 [timeseries.py:                 foo():182] [2088] 5200 frames saved
    24236910 [timeseries.py:                 foo():182] [2088] 5400 frames saved
